In [ ]:
from csc4585 import flour, java

# Introductory Walkthrough to Flour
This text assumes a background in Java.

## Hello, world
Hello world is a one-liner.

In [ ]:
%%flour

println("Hello, world!");

Unlike Java, Flour does not require your program to be contained within a class, nor does it have the concept of a main() function.

If, for whatever reason, you want to emulate a main() function, you need to call that function in the top level.

In [ ]:
%%flour

fn main() {
    println("Hello, world!");
}

main(); // We won't see any output without calling main() explicitly.

Like in Java, functions in Flour have return types. Our `fn main()` from the
previous example did not declare a return type, so it is assumed to be `void`.

In Java, `void` is a special case within the type system; it's not really a type,
it just indicates the absence of a return value. In Flour, `void` is a proper
type called a "unit type". This means we can have values of type `void`.

In [ ]:
%%java

public class Main {
    static void foo() { }
    
    public static void main(String[] args) {
        void val = foo(); // Nonsense in Java
    }
}

In [ ]:
%%flour

fn foo() { }

let val = foo(); // Perfectly legal in Flour!

This is actually a useful property, and it will matter more later.

As you may have noticed in the previous snippet, variables in Flour are declared with the `let` keyword. Like the `var` keyword in newer versions of Java, the type of the variable is inferred automatically. However, if you still want to declare the type explicitly, that's still possible.

In [ ]:
%%flour

// We can infer that `five` is an int.
let five = 5;

// We say explicitly that `six` is an int.
let six: int = 6;
    
// This syntax is equivalent to the one above; use whichever one you prefer.
let int seven = 7;

Unlike in Java, variables in Flour are not allowed to be uninitialized. If you do not explicitly assign a value in a `let` clause, a reasonable default will be inferred based on the variable's type. 

If no such default exists for a given type, the program won't run.

In [ ]:
%%flour

let nothing;
println(nothing);

let zero: int;
println(zero);

## Getting to know Flour

You're more than welcome to add your own cells to this notebook and experiment with Flour yourself. Any cell beginning with the `%%flour` magic will run as a Flour script. Take advantage of this to write your own programs or tinker with the existing cells.

Most things, when passed to `println()` will produce friendly, easily-read output.

You can also use the `type` keyword to inspect the type of things.

In [ ]:
%%flour

let hw = "Hello, world!";
let five = 2 + 3;

println(type(hw));
println(type(five));
println(type(type(five)));
println(type(&println));

## Functions

What if we actually want to return something from a function in Flour?
There are two ways to declare a return type; both are equivalent, so just
use whichever one you find easier to read.

In [ ]:
%%flour

// More like Java, might become harder to read with more complex types:
fn int two() { 
    return 2;
}

// Less like Java, easier to read with more complex types:
fn three() -> int {
    return 3;
}

The `fn` keyword is always required, and is always the very first part
of a standalone function declaration.

Flour also supports anonymous functions, like Java's lambdas. Unlike
lambdas in Java, which are sugar for anonymous inner classes, anonymous
functions in Flour are first-class functions.

Again, there are two ways to write the signature of an anonymous function,
and both are equivalent.

In [ ]:
%%flour

// Syntax 1:
let addFive = int(int a) {
    return a + 5;
};

// Syntax 2:
let addSix = (a: int) -> int {
    return a + 6;
};

(addFive(1) + addSix(2)).println();

What's going on in the final line? Flour supports a syntax called UFCS
(Uniform Function Call Syntax), which can be understood with the following
simple rule:

```
a.f(b, c);

is equivalent to

f(a, b, c);
```

This is a useful property for several reasons. But it is important to remember
that, unlike in Java, this dot-based function call syntax doesn't imply
any special relationship between the function and the receiver.

In [ ]:
%%flour

let addSix = (a: int) -> int {
    return a + 6;
};

5.addSix().println();

## Expressions and Control Flow
You should be familiar with the concept of expressions in Java.
Expressions in Flour work similarly, but more types of things can be
expressions.

For example, basic control flow structures like conditionals and loops can
be expressions in Flour. We call this "Block Value" (BV).

Observe the following Java program, demonstrating an `if`, a `while`, and a `for`. In the cells that follow, we will look at those structures in Flour.

In [ ]:
%%java

import java.util.ArrayList;

public class Main {
    public static void main(String[] args) {
        int val;

        if (2 + 3 == 5) {
            val = 10;
        } else {
            val = 20;
        }
        
        System.out.printf("val = %d\n", val);

        ArrayList<Integer> lst = new ArrayList<>();
        while (val <= 100) {
            lst.add(val);
            val += 10;
        }
        
        System.out.printf("lst = %s\n", lst);


        int target = 50;
        boolean found = false;
        for (int i : lst) {
            if (i == target) {
                found = true;
                break;
            }
        }
        
        System.out.printf("found = %b\n", found);
    }
}

In Flour, an expression standing by itself at the end a block becomes the value of that block.

This is the simplest application of Block Value.

In [ ]:
%%flour

let five = {
    println("2 + 3 is a standalone expression at the end of this block...");
    println("So it'll become the overall value of the block.");
    println("");
    
    2 + 3
};

println("Five is ${five}.");

This allows an `if` expression to yield different values depending on which branch is taken.

In [ ]:
%%flour

let val = if 2 + 3 == 5 { 
    10    
} else {
    20
};

println("val = ${val}");

The "default" value of any block is `void`. That means, if we don't explicitly give a value to something like an `if`, it will yield a `void`. This is a useful property; for example, we can often leave off the `else` from what would otherwise be a compound if-else expression.

In [ ]:
%%flour

let a = 5;
let b = -2;

let sqrtA = if a >= 0 {
    a ^ 0.5
};

let sqrtB = if b >= 0 {
    b ^ 0.5
};

println("sqrt(${a}) = ${sqrtA}");
println("sqrt(${b}) = ${sqrtB}");

Sometimes, when dealing with a value that might be `void`, we want to substitute a "default" value in place of `void`.

This is accomplished with the `~` squiggle operator.

(Yes, in this case, `else` also works just fine. We will soon look at some cases where `~` is more obviously the right choice.)

In [ ]:
%%flour

let a = 5;
let b = -2;

let sqrtA = if a >= 0 {
    a ^ 0.5
} ~ "Can't sqrt a negative number!";

let sqrtB = if b >= 0 {
    b ^ 0.5
} ~ "Can't sqrt a negative number!";

println("sqrt(${a}) = ${sqrtA}");
println("sqrt(${b}) = ${sqrtB}");

When working with loops (`while`, `loop`, and `for`), there are two ways to yield a value: `push` and `break`.

Loops maintain an internal list which can be appended to with `push`. This internal list becomes the overall value of the loop, unless the list is empty.

In [ ]:
%%flour

let val = 10;

let lst = while val <= 100 {
    push val;
    val = val + 10;
};

println("lst = ${lst}");

If you use a plain `break;` to exit the loop early, any `push`ed elements are preserved.

In [ ]:
%%flour

let bound = 5;
let i = 0;

let lst = loop {
    if i >= bound {
        break; // Plain `break` preserves the internal list
    }
    
    push i;
    i = i + 1;
};

println(lst);

If no `push`es occurred, the default value of `void` is used, *not* the empty list.

In [ ]:
%%flour

let foo = loop { 
    break; 
};

println(foo); // No `push`es occurred, so `foo` is void.

let bar = loop { 
    break; 
} ~ []; // We can use `~` if we want an empty list in this case.

println(bar);

When loops are nested, `push` always targets the "nearest" loop. To specify exactly which level `push` should target, you can use a label.

In [ ]:
%%flour
// TODO: This example needs the 2023-08-19 build of Flour!

let tops = ["Pullover", "Tshirt", "Sweater"];
let bottoms = ["Jeans", "Khakis", "Shorts"];

let pairs = for i in tops @outer {
    for j in bottoms {
        push @outer (i, j); // Explicitly push to `for i in...` marked with the @outer label
    }
};

for pair in pairs {
    println(pair);
}

You could also just use multiple nested `push` statements to propagate elements up several levels, though this will result in a nested list.

In [ ]:
%%flour

let tops = ["Pullover", "Tshirt", "Sweater"];
let bottoms = ["Jeans", "Khakis", "Shorts"];

let pairs = for i in tops {
    push for j in bottoms {  // This push targets `for i in...`
        push (i, j);         // This push targets `for j in...`
    };
};

for pair in pairs {
    println(pair);
}

As an aside: if you have a nested list which you want to flatten out, you could use the `flatMap()` function from the `lists` library.

In [ ]:
%%flour

import lists;

let tops = ["Pullover", "Tshirt", "Sweater"];
let bottoms = ["Jeans", "Khakis", "Shorts"];

let pairs = for i in tops {
    push for j in bottoms {  // This push targets `for i in...`
        push (i, j);         // This push targets `for j in...`
    };
}.flatMap();                 /* Since we had nested `push`es, we end up with nested lists.
                                lists::flatMap() will flatten the list for us. */

for pair in pairs {
    println(pair);
}

We can also use `break` to yield a value from a loop. Unlike a plain `break`, `break`ing with a value will override anything appended with `push`.

In [ ]:
%%flour

let target = 50;
let lst = [10..=100 by 10]; // Just carrying over from a previous cell...

let found = for i in lst {
    if i == target {
        break true;
    }
} ~ false; // In the case where we never hit `break true;`, replace `void` with `false`.

println("found = ${found}");

`break` can target a label in the same way as `push`.

In [ ]:
%%flour
// TODO: This example needs the 2023-08-19 build of Flour!

let tops = ["Pullover", "Tshirt", "Sweater"];
let bottoms = ["Jeans", "Khakis", "Shorts"];

let sweaterAndJeans = for i in tops @outer {
    for j in bottoms {
        if (i, j) == ("Sweater", "Jeans") {
            break @outer (i, j);
        }
    }
};

match sweaterAndJeans {
    (string, string) pair -> "Found pair! " + pair,
    _                     -> "Didn't find the pair."
}.println();

Notice that only one style of `for` loop is available. It is similar to the "enhanced for-loop" in Java.

In [ ]:
%%flour

let lst = [1, 2, 3];
for element in lst {
    println(element);
}

Recall that when an expression is sitting by itself at the end of a block `{ }`, it becomes the value of that block. This also extends to the return values of functions.

In [ ]:
%%flour

fn add(a: int, b: int) -> int {
    a + b
}

println(add(2, 3));

For maximum brevity, you could also use fat-arrow `=>` syntax. This only works with a single standalone expression.

In [ ]:
%%flour

fn add(a: int, b: int) -> int => a + b

println(add(2, 3));

## Unions (and some other relevant types)
An important concept in Flour is the *union*, a type which can represent one of a given set of types.

In [ ]:
%%flour

fn foo(b: bool) -> int|string {
    return if b {
        5
    } else {
        "five"
    };
}

foo(true).println();
foo(false).println();

println("");

type(foo(true)).println();
type(foo(false)).println();

In the above example, the function `foo()` can return either an integer or 
a string. You can think of the type `int|string` as "int OR string".

There are a few ways to deal with union types, like `match`.
A `match` statement functions a bit like Java's `switch` statement, but with
an important difference: while `switch` chooses a path based on the value's
identity, `match` chooses a path based on the value's *type*.

Like the other control flow statements, `match` can potentially
yield a value.

In [ ]:
%%flour

fn foo(b: bool) -> int|string {
    return if b {
        5
    } else {
        "five"
    };
}

match foo(false) {
    string strValue -> {
        println("Got a string: ${strValue}");
    },
    int intValue -> {
        println("Got an int: ${intValue}");
    }
}

let n = foo(true);
print("Square of ${n}: ");
match n {
    int intValue -> intValue * intValue,
    _            -> -1
}.println();

`_` means "anything else", like `default` in a Java switch statement.

When using an `if` expression as a value, the default value is `void` if no `else` branch is present.

In [ ]:
%%flour

let func = (int n) {
    return if n > 0 {
        n * 3
    };
};

println(func(2));
println(func(-5));

The type `*` is special. It's known as "wild", and it can represent any type. It is at the top of the type hierarchy, like Java's `Object`.

In [ ]:
%%flour

fn identity(x: *) -> * {
    return x;
}

println(identity("foo"));
println(identity(5));

The `*` type should only be used when there are no other alternatives, which is a situation that almost never arises.

You might think about using `*` to write functions and data structures that work on arbitrary types of input. As we'll see later, Flour offers a much better mechanism for writing generic code.

If you're curious, this is the proper way of writing a function like `identity()`, using a mechanism similar to generic types in Java:

In [ ]:
%%flour

fn [T] identity(x: T) -> T {
    return x;
}

println(identity("foo"));
println(identity(5));

#### Exercise
Correctly annotate the return type of the function `foo()`. The type cannot include `*`.

In [ ]:
%%flour

// Starter code:
fn foo(int a, string b) -> ??? {
    return if a < -5 {
        a * 7
    } else if a > 5 {
        b
    };
}

// Driver code:
foo(0, "foo").println();
foo(16, "foo").println();
foo(-20, "foo").println();

In Java, any (reference) type can also hold a `null` value.
This isn't possible in Flour, which doesn't have the concept of null. Instead,
if we want to express the type of a value that might be absent, we use a
union of that type with `void`.

In [ ]:
%%java

public class Main {
    static String[] digits = {
            "zero", "one", "two", "three", "four", 
            "five", "six", "seven", "eight", "nine"
    };
    
    static String digit(int d) {
        if (d < 0 || d > 9) {
            return null;
        } else {
            return digits[d];
        }
    }
    
    public static void main(String[] args) {
        System.out.println(digit(7));  // "seven"
        System.out.println(digit(55)); // null
    }
}

In [ ]:
%%flour

let digits = [
        "zero", "one", "two", "three", "four",
        "five", "six", "seven", "eight", "nine"
];

fn digit(d: int) -> string|void {
    if d < 0 || d > 9 {
        return (); // `()` represents the value of type `void`.
    } else {
        return digits[d];
    }
}

println(digit(7));  // "seven"
println(digit(55)); // void


#### Exercise
Below, find a partially-completed version of the `digit()` function which is written with Block Value syntax.
Try to complete the function, replacing instances of `???`.

Remember that the default value of an `if` expression is `void`.

In [ ]:
%%flour

// Starter code:

fn digit(d: int) -> string|void {
    return if ??? {
        ???
    };
}

// Driver code:
    
let digits = [
        "zero", "one", "two", "three", "four",
        "five", "six", "seven", "eight", "nine"
];

println(digit(7));  // "seven"
println(digit(55)); // void

Unions can be any arity above 1, so a type such as `int|float|string|void`
(int OR float OR string OR void) is legal,
even if clumsy.

As an aside, the built-in type `number` represents the union `int|float|real`.


## Product Types (Data Structures)
Consider the following class in Java.

In [ ]:
%%java

public class Student {
    
    public String name;
    public int id;

    public Student(String name, int id) {
        this.name = name;
        this.id = id;
    }

    public static void main(String[] args) {
        Student s = new Student("Ada Lovelace", 11223344);
        System.out.printf("Name: %s, Id: %d", s.name, s.id);
    }
}

Flour does not have classes. Instead, we can represent this sort of product
type in one of two ways: using tuples, or using tables.

### Tables

Tables associate names with values.

In [ ]:
%%flour

let t = {
    name -> "Ada Lovelace",
    id   -> 11223344
};

println(t);
println("Name: " + t.name);

If you want to define a type of table
which holds specific values, you define an *archetype*. An instance of 
a table which is associated with an archetype is called an *object*.
This brings us closer to classes in Java.

In [ ]:
%%flour

archetype Student {
    name: string,
    id:   int
}

let t = Student {
    name -> "Ada Lovelace",
    id   -> 11223344
};

println(t);
println("Name: " + t.name);

#### Exercise
Try translating the following Java class into a Flour archetype (don't worry about main).

In [ ]:
%%java

public class Airplane {
    public String company;
    public int model;
    public float price;
    
    public Airplane(String company, int model, float price) {
        this.company = company;
        this.model = model;
        this.price = price;
    }
    
    public static void main(String[] args) {
        Airplane jane = new Airplane("Boeing", 747, 200000000.00f);
        System.out.printf("%s %d %f", jane.company, jane.model, jane.price);
    }
}

In [ ]:
%%flour

// Starter code:
archetype Airplane {
    ???
}

// Driver code:
let jane = Airplane {
    company -> "Boeing",
    model   -> 747,
    price   -> 200000000.00
};

println("${jane.company} ${jane.model} ${jane.price}");

Defining an archetype also allows us to define member functions.

In [ ]:
%%flour

archetype Student {
    name: string,
    id:   int
}

fn<Student> getInfo() -> string {
    "Name: ${this.name}, Id: ${this.id}"
}

let t = Student {
    name -> "Ada Lovelace",
    id   -> 11223344
};

println(t.getInfo());

It's also possible to define a constructor. 
Unlike in Java, constructors are nothing special. They simply provide a shorthand to
define a function with the same name and return type as the archetype.

In [ ]:
%%flour

archetype Student {
    name: string,
    id:   int
}

constructor<Student>(string name, int theId) {
    // We don't need to declare the table as being a `Student`, it's inferred since we're in the constructor
    return { 
        $name,       // $name is the same as name -> name
        id -> theId
    }; 
}

// This is an even shorter form of constructor which allows us to skip the nested pair of `{}`
constructor<Student>() {
    name -> "John Doe",
    id   -> 9999
}

fn<Student> getInfo() -> string {
    "Name: ${this.name}, Id: ${this.id}"
}

let t = Student("Ada Lovelace", 11223344); // Note the lack of `new`
let u = Student();

println(t.getInfo());
println(u.getInfo());

### Tuples
Tuples offer a different approach towards product types, more akin to a
functional programming style. Instead of binding names to values, tuples
use indices like lists. Unlike lists, tuples cannot change in size, and are
immutable (i.e. their contents cannot change).

In their most basic form, tuples are useful for tasks such as returning
multiple values from a function.

In [ ]:
%%flour

fn quotientAndRemainder(number x, number y) -> (number, number) {
    return (floor(x/y), x%y);
}

quotientAndRemainder(5, 2).println(); // (2, 1)

let qr = quotientAndRemainder(6, 4);
println("remainder: ${qr.1}"); // remainder: 2

let (quot, rem) = quotientAndRemainder(18, 4);
println("15/4 = ${quot} R${rem}"); // 15/4 = 4 R2

#### Exercise
Complete the sqrt() function. The function should return a tuple of two numbers: the real and imaginary parts of one complex number.

In [ ]:
%%flour

fn sqrt(number x) -> (number, number) {
    return if x >= 0 {
        ???
    } else {
        ???
    };
}

// Driver code:
println(sqrt(16));
println(sqrt(-9));

This is how we might represent our Student type from before as a tuple.

In [ ]:
%%flour

typedef Student = (string, int);

fn Student(string name, int id) -> Student {
    return (name, id);
}

fn name(Student s) -> string {
    return s.0;
}

fn id(Student s) -> int {
    return s.1;
}

fn getInfo(Student s) -> string {
    return "Name: ${s.name()}, Id: ${s.id()}";
}

let t = Student("Ada Lovelace", 11223344);

println(t.getInfo());

The first line contains a `typedef`, a mechanism which allows us to refer to a complex type by a given name.

Notice how the final two lines, the code which actually interacts with our type,
are unchanged from the previous example using `archetype`.
While tuples and tables are fundamentally different, they can accomplish
many of the same abstractions. In general, archetypes are a bit easier to set
up and understand, but tuples have more capabilities.

## I/O
`print()` and `println()` work as you'd expect in Java.

In [ ]:
%%flour

print("1 + 2 = ");
println(3);

`printf()` does not exist, but string interpolation can accomplish the
same task.
Anything enclosed in `${}` within a string literal will be evaluated as a 
Flour expression and substituted into the string.

In [ ]:
%%flour

println("1 + 2 = ${1+2}");

`readln()` gets a line of input from stdin.

However, due to limitations imposed by the Jupyter notebook, using `readln()` within a Flour cell is impossible.
We can partially solve this problem with the magical `${input}` directive, used as follows:

In [ ]:
%%flour

println("Intro.");

let foo = ${input("Type something")};
let foobar = ${input("Type something else")};

println("You typed: `${foo}` and also `${foobar}`");

println("Outro.");

This directive is processed before the program runs, meaning we cannot interactively get input from the user; we must get it all at once before any Flour code can execute. Notice how, in the above example, `"Intro"` was not printed to the screen until input was provided.

In [ ]:
%%flour

let name = ${input("Type your name")};

let typedAge = ${input("Type your age")};
let age = typedAge.parseInt() as! int;

println("Name: ${name}, Age: ${age}");

`parseInt()` takes a string, and returns `int|void`. If the string isn't
recognized as a valid integer, the `void` value is returned. By using
`as!` here, we are asserting that `parseInt()` will return an `int`, not `void`.
If this turns out not to be the case, the program will crash.

Since the user might type a bad value when prompted for the age, we should
probably handle that case explicitly.

Were we not limited by Jupyter, we might even add a loop which keeps asking the user to retry until an acceptable input is provided.

In [ ]:
%%flour 

let name = ${input("Type your name")};

let age = match ${input("Type your age")}.parseInt() {
    int age -> age,
    _       -> println("Invalid age was provided.")
};

println("Name: ${name}, Age: ${age}");

We could also assume a default value with the `~` operator.

In [ ]:
%%flour

let name = ${input("Type your name")};

let age = ${input("Type your age")}.parseInt() ~ 0;

println("Name: ${name}, Age: ${age}");

#### Exercise

Using `for`, `match`, and `push`, produce a new list which contains all the elements of `lst` which are not `void`.
Replace `???` with your own code.

In [ ]:
%%flour

// Starter code below //
let lst = [1, 3, 5, (), 12, (), (), 16, 27];

let filteredLst = for num in lst {
    ???
};

// Driver code below //
println(filteredLst);
println(filteredLst == [1, 3, 5, 12, 16, 27]);

## Lists
Lists work largely the same as Java's ArrayList, but you can use `[]` syntax for indexing, like Java's arrays. `add()`, `remove()`, `contains()`, and other basic ArrayList operations are available.

You can use list literal syntax to create a list. For example, the expression `[1, 2, 3]` creates a list containing those three integers.

Lists always have a specific type, such as `list<string>` for a list containing only `string`s. The type of a list can usually be inferred by the language when using list literal syntax.

You may sometimes need to specify the type of a list when creating it, especially when the list is initially empty. For example, `<int>[]` creates an empty `list<int>`.

You can use range syntax `[x..y]` to quickly create a list with ascending numbers between `x` and `y`.
This syntax is also useful for `for` loops when you want to iterate a set number of times.

You can use the *put* operator `<-` to quickly append to a list.
You can concatenate two lists with `+`.

In [ ]:
%%flour

let lst = [1, 2, 3];
println(lst);

lst = [10..20];
println(lst);

lst[1] = 5;
println(lst);

lst.remove(0);
println(lst);

lst.add(99, 0);
println(lst);

lst <- 55;
lst <- 66;

println(lst);
println("Length: " + lst.len());

println([1..5] + [5..=10]);

#### Exercise
Write a function, `sortList()`, that accepts a `list<int>` and returns a new list, with the contents sorted.
Use any sorting algorithm you like. You're free to write additional helper functions.

In [ ]:
%%flour

// Starter code:
fn sortList(li: list<int>) -> list<int> { 
    ???
}

// Driver code:
sortList([5, 3, 1, 6, 10, 2, 4, 9, 7, 8]).println();

## Defer
`defer` delays execution of a statement until the current scope is left. This is most useful for cleaning up resources, for example unlocking a mutex at the end of a scope, because the deferred code will always run regardless of *how* control flow exits the current scope.

In [ ]:
%%flour

import parallel;

let m = mutex(0);

fn increment(m: mutex<int>) {
    m.lock();
    defer m.unlock();
    m.set(m.get() + 1);
}

increment(m);
increment(m);

m.lock();
println(m.get());

`defer` can be used to accomplish a similar effect to Java's try/finally.

If Java had a mutex API resembling Flour's, the preceding cell might look like this in Java:

In [ ]:
%%java

import java.util.concurrent.locks.ReentrantLock;

class Mutex<T> {
    ReentrantLock _lock = new ReentrantLock();
    T value;
    public void lock() { _lock.lock(); }
    public void unlock() { _lock.unlock(); }
    public T get() { return value; }
    public void set(T i) { value = i; }
    public Mutex(T i) { value = i; }
}

public class Main {
    static void increment(Mutex<Integer> m) {
        try {
            m.lock();
            m.set(m.get() + 1);
        } finally {
            m.unlock();
        }
    }
    
    public static void main(String[] args) {
        Mutex<Integer> m = new Mutex<>(0);
        increment(m);
        increment(m);
        
        m.lock();
        System.out.println(m.get());
    }
}

## More About Functions
Flour supports first-class functions. In practical terms, that means you can pass them around like any other type of value.

In [ ]:
%%flour

fn createAdder() -> (int, int) -> int {
    return (int a, int b) -> int {
        return a + b;
    };
}

let adderFunc = createAdder();

adderFunc.println();
adderFunc(3, 5).println();

createAdder() is a higher-order function. It is a function which returns another function.

You can reference a normal named function like a variable by prefixing its name with `&`.

In [ ]:
%%flour

fn addNums(a: int, b: int) -> int {
    return a + b;
}

let adder = &addNums;

println(adder);
println(adder(3, 5));

Higher-order functions can also take functions as arguments. Imagine we wanted to write a method whose behavior we might want to customize. For example, a `filter()` function which discards elements in a list based on some predicate. We could pass a function to `filter()` which defines the predicate.

#### Exercise
Try to complete the `filter()` function below.

In [ ]:
%%flour

// Given code:
fn filter(listToFilter: list<int>, shouldWeKeepThis: (int) -> bool) -> list<int> {
    ???
}

//Driver code:
let li = [1..=20];
let onlyOddNums = li.filter(bool(int a) => a % 2 != 0);
println(onlyOddNums);

## Closures
Anonymous functions have the ability to "capture" variables around them for later use, even when those variables are out of scope by the time the function is called. This combination of a function and its captured environment is called a closure.

In [ ]:
%%flour

fn createAdder(int a) -> (int) -> int {
    return (int b) -> int {
        return a + b;
    };
}

let addFive = createAdder(5);

addFive.println();
addFive(3).println();

createAdder(40)(25).println();

Even though `a` has gone out of scope by the time we call `addFive(3)`, it still "remembers" what `a` was bound to.

It turns out that closures are all you need to implement classes and objects. Iterators and parallel data structures, for example, are implemented in Flour with closures.

The following two cells are essentially equivalent.

In [ ]:
%%flour

// Archetype-style object.

archetype Generator {
    n: int,
    bound: int
}

constructor<Generator>(int n, int bound) {
    $n, $bound
}

fn<Generator> next() -> int|void {
    return if this.hasNext() {
        defer this.n = this.n + 1;
        this.n
    };
}

fn<Generator> hasNext() -> bool {
    return this.n < this.bound;
}

// Using our object:

let gen = Generator(0, 10);
while gen.hasNext() {
    println(gen.next());
}

In [ ]:
%%flour

// Closure-style object.

typedef Next      = () -> (int|void);
typedef HasNext   = () -> bool;
typedef Generator = (Next, HasNext);

fn Generator(int n, int bound) -> Generator {
    let t = {$n};

    let hasNext = () -> bool {
        return t.n < bound;
    };

    let next = () -> int|void {
        return if hasNext() {
            defer t.n = t.n + 1;
            t.n
        };
    };

    (next, hasNext)
}

fn next(Generator g)    -> int|void => (g.0)()
fn hasNext(Generator g) -> bool     => (g.1)()

// Using our "object":

let gen = Generator(0, 10);
while gen.hasNext() {
    println(gen.next());
}

Note that closures perform capture *by value,* not by reference. This means you need to take care when you want a closure to have access to a mutable value type, such as `int`.

In [ ]:
%%flour

// This code DOES NOT WORK how you might think!

typedef Add = () -> void;
typedef Get = () -> int;

fn Foo() -> (Add, Get) {
    let n = 0;
    return (
        void() { n = n + 1; },
        int()  { n }
    );
}

let (add, get) = Foo();

add();
add();

println(get());

The easiest solution is to wrap value types in a reference type, such as a table.

In [ ]:
%%flour

typedef Add = () -> void;
typedef Get = () -> int;

fn Foo() -> (Add, Get) {
    let t = {n -> 0};
    return (
        void() { t.n = t.n + 1; },
        int()  { t.n }
    );
}

let (add, get) = Foo();

add();
add();

println(get());

## Generics
Let's take another look at our `filter()` function from earlier.

In [ ]:
%%flour

fn filter(listToFilter: list<int>, shouldWeKeepThis: (int) -> bool) -> list<int> {
    return for e in listToFilter {
        if shouldWeKeepThis(e) {
            push e;
        }
    };
}

This is fine, but what if we wanted to apply this function to any kind of list, not just `list<int>`?

We could try swapping `int` for `*`.

In [ ]:
%%flour

fn filter(listToFilter: list<*>, shouldWeKeepThis: (*) -> bool) -> list<*> {
    return for e in listToFilter {
        if shouldWeKeepThis(e) {
            push e;
        }
    };
}

let a = [1..=10];
let evens = a.filter(bool(*i) => i % 2 == 0);

let b = ["foo", "bar", "baz"];
let bWords = b.filter(bool(*s) => s.char(0) == 'b');

println(evens);
println(bWords);

At a glance, this does work. However, on top of destroying type information, it allows us to write nonsense.

In [ ]:
%%flour

fn filter(listToFilter: list<*>, shouldWeKeepThis: (*) -> bool) -> list<*> {
    return for e in listToFilter {
        if shouldWeKeepThis(e) {
            push e;
        }
    };
}

let b = ["foo", "bar", "baz"];
let evens = b.filter(bool(*i) => i % 2 == 0);

This crashes, of course. We know by looking at this code that it wouldn't work -- there's no way to take the modulo of a string -- but the language can't know that ahead of time. By using `*`, we have lost the ability to express the relationship between the function's parameters and its output. In this case, `*` really can be "any type", but it also must be the *same* type in all 3 places.

We solve this problem with generics.

In [ ]:
%%flour

fn [T] filter(listToFilter: list<T>, shouldWeKeepThis: (T) -> bool) -> list<T> {
     return for e in listToFilter {
        if shouldWeKeepThis(e) {
            push e;
        }
    };
}

let a = [1..=10];
let evens = a.filter(bool(int i) => i % 2 == 0);

let b = ["foo", "bar", "baz"];
let bWords = b.filter(bool(string s) => s.char(0) == 'b');

println(evens);
println(bWords);

By adding a type parameter `T`, we can now properly express the relationship between all the types it appears in, while still allowing the function to be used on lists of any type.

Of course, we might need multiple type parameters for a single function. Consider `mapOpt()` from the `misc` library.

In [ ]:
%%flour

fn [T, R] mapOpt(input: T|void, mappingFn: (T) -> R) -> R|void {
    return match input {
        T t -> mappingFn(t),
        _ -> ()
    };
}

let five  = 5;
let empty = ();
    
let addTen = int(int a) => a + 10;

five.mapOpt(addTen).println();
empty.mapOpt(addTen).println();

This function accepts a possibly-absent value, applies a transformation to the value if it exists, then returns the result. Using `T` and `R`, we can properly express the relationship between the mapping function, the input, and the output.

The input is of type `T|void`, the union of any arbitrary type and `void`. 
The output is of type `R|void`, the union of another arbitrary type and `void`.
The relationship between these two types is brought together by the mapping function, `(T) -> R`. The return type of this function determines the return type of `mapOpt`, and the parameter type must agree with the supplied input.

#### Exercise
Consider the generic sorting algorithm below, and fill in the `???` with the correct type information.

In [ ]:
%%flour

// Given code:
fn [T] sortBy(li: ???, comp: ???) {
    let len = len(li);
    
    for i in [0..len] {
        let minI = i;
        
        for j in [i+1..len] {
            if comp(li[j], li[minI]) < 0 {
                minI = j;
            }
        }
        
        li.swap(i, minI);
    }
}

// Driver code:
fn [T] swap(l: list<T>, i: int, j: int) {
    let tmp = l[i];
    l[i] = l[j];
    l[j] = tmp;
}    

let la = [4, 3, 6, 5, 1, 9, 2, 8, 7];
let lb := la;

la.sortBy(int(int a, int b) => a - b);
lb.sortBy(int(int a, int b) => b - a);

let lc = ["ab", "bac", "abcde", "abcd", "a", "abc"];
let ld := lc;
    
lc.sortBy(int(string a, string b) => len(a) - len(b));
ld.sortBy(&compare);

println("Ascending:       " + la);
println("Decending:       " + lb);
println("Lengthwise:      " + lc);
println("Lexicographical: " + ld);

## Libraries
Using `import`, you can bring other Flour files into the current context, allowing you to call functions and utilize types defined within them.

In this notebook, importing custom libraries will be impossible, so we will not cover how to write your own library files. However, you are still able to import various pre-packaged library files which make up the Flour Standard Library.

Here are the most important libraries which are available for you to use:
* parallel - Provides support for launching threads, as well as the Fut and Channel types.
* lists - Provides various functions for operating on lists, such as map(), reduce(), and enumerate().
* iter - Provides support for iterators.

In [ ]:
%%flour

import iter;   // Bringing in the `iter` library.

let a = [  1,   2,   3,   4,   5];
let b = ["A", "B", "C", "D", "E"];

let ab = a.zipIter(b);    // iter::zipIter() returns an iterator over 2 lists, zipped together.

while ab.hasNext() {      // hasNext() and next() are member functions of an iterator.
    println(ab.next());
}

Iterators are implemented as closure-style objects.

This means that the type `Iter[T]` can be considered an interface, and an implementation of that interface
is defined by a function (called a constructor) whose name indicates the name of the (sub)type.

The built-in `Iter[T]()` constructor is, in fact, just one of many possible implementations of the `Iter[T]` interface.
If we want to implement this `Iter[T]` interface ourselves to create a new kind of iterator with different behavior,
we simply create a new constructor with a different name.

This way of thinking about objects is rather advanced, especially if you are coming from languages like
Java and C++ whose object-oriented paradigm much more closely resembles archetype-style objects.

In [ ]:
%%flour

import iter;

fn [T] InfiniteIter(init: T, nextFn: (T) -> T) -> Iter[T] {
    let final state = { i -> init };
    
    return (
        IterRet[T]() {   // next()
            defer state.i = nextFn(state.i);
            state.i
        },
        bool() => true   // hasNext()
    );
}

fn [N : number] InfiniteAscendingIter(init: N) -> Iter[N] => InfiniteIter(init, N(N a) => a + 1)

let itr = InfiniteAscendingIter(1.5);

for x in [0..15] {
    println(itr.next());
}

Here's a Java equivalent of what we just wrote:

In [ ]:
%%java

import java.util.*;
import java.util.function.*;

public class Main {
    
    static class InfiniteIter<T> implements Iterator<T> {
        private T i;
        private Function<T, T> nextFn;
        
        public InfiniteIter(T init, Function<T, T> nextFn) {
            this.i = init;
            this.nextFn = nextFn;
        }
        
        public boolean hasNext() {
            return true;
        }
        
        public T next() {
            T last = i;
            i = nextFn.apply(i);
            return last;
        }
    }
    
    static class InfiniteAscendingIter<N extends Number> extends InfiniteIter<Number> {
        public InfiniteAscendingIter(N init) {
            super(init, (Number a) -> {
                if (a instanceof Double || a instanceof Float) {
                    return a.doubleValue() + 1.0;
                } else {
                    return a.longValue() + 1;
                }
            });
        }
    }
    
    public static void main(String[] args) {
        InfiniteAscendingIter<Double> itr = new InfiniteAscendingIter<>(1.5);
        
        for (int x = 0; x < 15; ++x) {
            System.out.println(itr.next());
        }
    }
}

## Parallelism
Flour's parallelism paradigm revolves around the concept of "Shareability", which is enforced by a special type of closure called a "Share Closure". By strictly regulating how and when threads share memory, an entire class of parallel programming bugs becomes impossible to write in Flour.

### Share
Shareability is expressed with the `share` type attribute. If a type is `share`, that means Flour considers it safe for multiple threads to access it simultaneously.

The following types are always `share`:
* Value types (int, bool, string, etc.)
* Mutexes
* Atomics
* Share closures

The following types are `share` if all their members are `share`:
* Tuples
* Unions

Otherwise, any deeply immutable type is `share`.
Use of the `imut` type specifier may help with this.
For example, `list<int>` is not `share`, but `imut list<int>` is.

It is not necessary to annotate types as `share`; it is simply
a property of the type.

### Share Closures
Share Closures are written like anonymous functions, prefixed
with `share<...>` or `acquire<...>`, where `...` is a list of 
variables to capture. Only variables which satisfy the `share`
type attribute can be captured. Unlike with normal anonymous
functions, variable capture is strictly explicit; any symbol
not present in the `share<>` list will not be captured.

In [ ]:
%%flour

let m = mutex(0);

let incr = share<m> void() {
    m.lock();
    defer m.unlock();
    m.set(m.get() + 1);
};

incr();

m.lock();
println(m.get());

Share closures don't do anything special on their own. To actually run code in parallel, you need to pass a share closure to the `spawn()` function, found in the `parallel` library.

`spawn()` submits a task to the thread pool, then returns immediately. Whatever function you supply to `spawn()` will be automatically called and run in parallel.

`spawn()` returns a handle to the spawned task. You can call `join()` on this handle to block until the task is complete. If the provided task also returns a value, it will propagate up to `join()`.

`spawn()` will ONLY accept share closures; normal functions cannot be passed to `spawn()`.

In [ ]:
%%flour

import parallel;

let m = mutex(0);

let incr = share<m> void() {
    m.lock();
    defer m.unlock();
    m.set(m.get() + 1);
};

spawn(incr).join(); // Run incr() in a new thread, then wait for it to finish.

m.lock();
println(m.get());

In [ ]:
%%flour

import parallel;

let m = mutex(0);

let incr = share<m> void() {
    m.lock();
    defer m.unlock();
    m.set(m.get() + 1);
};

let get = share<m> int() {
    m.lock();
    defer m.unlock();
    m.get()
};

spawn(incr).join();

println(spawn(get).join());

Using `acquire<>` instead of `share<>` will automatically
acquire and release captured resources when the task is called.
Because `acquire<>` might need to wait for resources, calling a share closure
defined with `acquire<>` implies an arbitrary, possibly infinite 
delay before running. Resources will *not* be acquired until the function
is explicitly called, either by `spawn()` or manually.

In [ ]:
%%flour

import parallel;

let m = mutex(0);

spawn(acquire<m> void() {
    m.set(1); // Don't need to lock() or unlock()!
}).join();

m.lock();
println(m.get());